# Personal Task Assistant with Gradio - LangChain 1.0

A comprehensive example combining multiple capabilities.

**Features:**
- Task management (add, list, complete tasks)
- Simple calculations
- Date/time information
- Interactive Gradio interface

In [ ]:
# Install dependencies
!pip install --pre -U langchain langchain-openai gradio

In [ ]:
# Set up API key
import os
os.environ['OPENAI_API_KEY'] = 'your-api-key-here'  # Replace with your key

In [ ]:
from langchain_core.tools import tool
from typing import Annotated
from datetime import datetime, timedelta

# Simple in-memory task storage
tasks = []

# Task Management Tools
@tool
def add_task(task: Annotated[str, "Task description"],
             priority: Annotated[str, "Priority: high, medium, or low"] = "medium") -> str:
    """Add a new task to the task list."""
    task_id = len(tasks) + 1
    new_task = {
        "id": task_id,
        "task": task,
        "priority": priority.lower(),
        "completed": False,
        "created": datetime.now().strftime("%Y-%m-%d %H:%M")
    }
    tasks.append(new_task)
    return f"✅ Task added: '{task}' (Priority: {priority}, ID: {task_id})"

@tool
def list_tasks(filter_by: Annotated[str, "Filter: all, pending, or completed"] = "all") -> str:
    """List all tasks or filter by status."""
    if not tasks:
        return "No tasks found. Add some tasks to get started!"
    
    filter_by = filter_by.lower()
    filtered_tasks = tasks
    
    if filter_by == "pending":
        filtered_tasks = [t for t in tasks if not t["completed"]]
    elif filter_by == "completed":
        filtered_tasks = [t for t in tasks if t["completed"]]
    
    if not filtered_tasks:
        return f"No {filter_by} tasks found."
    
    result = [f"\n📋 Tasks ({filter_by}):"]
    for task in filtered_tasks:
        status = "✓" if task["completed"] else "○"
        priority_emoji = {"high": "🔴", "medium": "🟡", "low": "🟢"}[task["priority"]]
        result.append(
            f"{status} [{task['id']}] {priority_emoji} {task['task']} (Created: {task['created']})"
        )
    
    return "\n".join(result)

@tool
def complete_task(task_id: Annotated[int, "Task ID to mark as complete"]) -> str:
    """Mark a task as completed."""
    for task in tasks:
        if task["id"] == task_id:
            if task["completed"]:
                return f"Task {task_id} is already completed."
            task["completed"] = True
            return f"✅ Task {task_id} marked as complete: '{task['task']}'"
    return f"❌ Task {task_id} not found."

@tool
def delete_task(task_id: Annotated[int, "Task ID to delete"]) -> str:
    """Delete a task from the list."""
    for i, task in enumerate(tasks):
        if task["id"] == task_id:
            deleted_task = tasks.pop(i)
            return f"🗑️ Deleted task {task_id}: '{deleted_task['task']}'"
    return f"❌ Task {task_id} not found."

# Utility Tools
@tool
def calculate(expression: Annotated[str, "Mathematical expression to evaluate"]) -> str:
    """Calculate a mathematical expression."""
    try:
        # Safe eval for basic math
        allowed_chars = set('0123456789+-*/(). ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"
        result = eval(expression)
        return f"{expression} = {result}"
    except Exception as e:
        return f"Error calculating: {str(e)}"

@tool
def get_current_datetime() -> str:
    """Get the current date and time."""
    now = datetime.now()
    return f"""Current Date & Time:
📅 Date: {now.strftime('%A, %B %d, %Y')}
🕐 Time: {now.strftime('%I:%M:%S %p')}"""

@tool
def get_date_after_days(days: Annotated[int, "Number of days from today"]) -> str:
    """Get the date after a specified number of days from today."""
    target_date = datetime.now() + timedelta(days=days)
    return f"{days} days from now: {target_date.strftime('%A, %B %d, %Y')}"

print("✅ Task assistant tools created!")

In [ ]:
from langchain.agents import create_agent

# Collect all tools
assistant_tools = [
    add_task,
    list_tasks,
    complete_task,
    delete_task,
    calculate,
    get_current_datetime,
    get_date_after_days
]

# Create agent using LangChain 1.0 pattern
task_agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=assistant_tools,
    prompt="""You are a helpful personal assistant. You can:
    
    📝 Task Management:
    - Add tasks with priorities (high, medium, low)
    - List all tasks or filter by status
    - Mark tasks as complete
    - Delete tasks
    
    🔧 Utilities:
    - Perform calculations
    - Get current date and time
    - Calculate future dates
    
    Be helpful, friendly, and use the appropriate tools to assist the user.
    When listing tasks, always use the list_tasks tool."""
)

print("✅ Task assistant agent created!")
print(f"\nAvailable tools ({len(assistant_tools)}):")
for tool in assistant_tools:
    print(f"  - {tool.name}")

In [ ]:
# Test the agent
print("Test 1: Adding tasks")
result = task_agent.invoke({
    "messages": [{"role": "user", "content": "Add a high priority task: Finish project report"}]
})
print(result['messages'][-1].content)

print("\n" + "="*70 + "\n")

print("Test 2: Adding more tasks")
result = task_agent.invoke({
    "messages": [{"role": "user", "content": "Add two more tasks: 'Buy groceries' (medium priority) and 'Call dentist' (low priority)"}]
})
print(result['messages'][-1].content)

print("\n" + "="*70 + "\n")

print("Test 3: Listing tasks")
result = task_agent.invoke({
    "messages": [{"role": "user", "content": "Show me all my tasks"}]
})
print(result['messages'][-1].content)

In [ ]:
import gradio as gr

def chat_with_assistant(message, history):
    """Process user message and return assistant response."""
    try:
        result = task_agent.invoke({
            "messages": [{"role": "user", "content": message}]
        })
        return result['messages'][-1].content
    except Exception as e:
        return f"Error: {str(e)}"

# Create Gradio interface
demo = gr.ChatInterface(
    fn=chat_with_assistant,
    title="🤖 Personal Task Assistant - LangChain 1.0",
    description="""Your AI assistant for task management and daily utilities!
    
    **I can help you:**
    - 📝 Manage tasks (add, list, complete, delete)
    - 🧮 Perform calculations
    - 📅 Check dates and times""",
    examples=[
        "Add a high priority task: Review presentation slides",
        "Show me all my pending tasks",
        "Mark task 1 as complete",
        "What's today's date?",
        "Calculate 15% of 240",
        "What date is it 30 days from now?",
        "List all my high priority tasks"
    ],
    theme=gr.themes.Soft()
)

# Launch the interface
demo.launch()

## Key Features

### 1. Multiple Tool Categories
- **Task Management**: Full CRUD operations
- **Utilities**: Calculations and date/time

### 2. State Management
- In-memory task storage
- Persistent across invocations in same session

### 3. LangChain 1.0 Pattern
```python
agent = create_agent(
    model="openai:gpt-4o-mini",
    tools=tools,
    prompt="System instructions"
)

result = agent.invoke({
    "messages": [{"role": "user", "content": message}]
})
```

### 4. Tool Design Best Practices
- Clear, descriptive names
- Detailed docstrings
- Type hints with Annotated
- Error handling
- User-friendly responses

## Example Conversations

**User:** "Add three tasks: finish homework (high), buy milk (low), and call mom (medium)"
**Agent:** Uses `add_task` three times, creating all tasks with proper priorities.

**User:** "Show my pending high priority tasks"
**Agent:** Uses `list_tasks` with appropriate filters.

**User:** "What's 15% of 350?"
**Agent:** Uses `calculate` tool: "0.15 * 350 = 52.5"

## Extensions

- Add due dates to tasks
- Task categories/tags
- Reminders
- Export tasks to file
- Calendar integration
- Email notifications

## Production Considerations

For production use:
1. Replace in-memory storage with database
2. Add user authentication
3. Implement proper error handling
4. Add data validation
5. Use environment variables for API keys
6. Add logging and monitoring